# Using Qasm in Qiskit

You can write quantum programs in qasm quantum assembly code.

We say "qasm" meaning the currently supported quantum assembly language.

See:
* [OPENQASM 2.x](https:arxiv.org/abs/1707.03429)
* [Qiskit/openqasm: Gate and operation specification for ... - GitHub](https://github.com/Qiskit/openqasmhttps://github.com/Qiskit/openqasm)

You can use Qiskit to execute your quantum programs on any supported backend.


## Backends

Backends are environments which run your code.

All of the "real" work of quantum processing takes place "remotely". Here, "remotely" may possibly mean "in the Qiskit Aer simulator on the user's own workstation". In that case, communication still takes place in the same communication model, whether it's a local simulator, a remote simulator in the cloud, or a remote IBM Q quantum processing unit in the cloud.

Generally the backends you encounter will be one of the following:
* the Qiskit Aer simulator running on your local workstation
* the [IBM Q](https://quantum-computing.ibm.com) simulator running in the cloud
* one of the [IBM Q](https://quantum-computing.ibm.com) cloud-hosted quantum processors

Qiskit provides an abstract backend and communicates with backend implementations via protocols, such as [JSON](https://www.json.org/) documents.

### Custom backends
Qiskit is open source: anyone can write a backend implementation and host their own services reachable by Qiskit code.

A backend may be
* a simulator
* an IBM Q (or other) quantum processor
* an abacus operated by a clockwork automaton
* a little old person in a backroom with paper and a pencil.

The first two are implemented by Qiskit and IBM Q Experience. The latter two exotic possibilities mentioned above can be implemented by you or others when you create new backend implementations.

## Primary Relevant Qiskit Components

* [Qiskit Terra](https://github.com/Qiskit/qiskit-terra)
  * qasm translation
  * circuit generation
  * communications to backend
    * abstract backend
* [Qiskit IBMQ Provider](https://github.com/Qiskit/qiskit-ibmq-provider)
  * qiskit-ibmq-provider backend implementation for communications to IBMQ backend

## Qasm and Circuit

Qasm quantum assembly language is a theoretical gate model which evolved independently of IBM in the quantum computing community.

Theorists express quantum operations using linear algebraic expressions.

Qasm high-level operators represent quantum operations taking arguments of register and other types.

These textual specifications of operations are rendered into [circuit](https://github.com/Qiskit/qiskit-tutorials/blob/master/qiskit/terra/visualizing_a_quantum_circuit.ipynb) form. Those circuits are then [transpiled](https://github.com/Qiskit/qiskit-tutorials/blob/master/qiskit/terra/using_the_transpiler.ipynb) into a sequence of low-level operations of typically limited variety offered by the target backend.

Qasm can express circuits and circuits can express qasm.

Typically, your Qiskit program creates a Circuit from your qasm code. Transpilation happens automatically. Then your Qiskit program submits the circuit derived from your qasm code to the backend as a Job for execution.

Programmable levels exist below the higher level of qasm. Such matters are part of [OpenPulse](https://arxiv.org/pdf/1809.03452.pdf).

### Transpilation

When qasm programs are **transpiled** (compiled with transformation) for backend consumption, they may be transpiled into longer or shorter strings of lower level operators called "basis gates", i.e., the limited set of operators physically or virtually implemented at the next level which will be receiving the transpilation output.

Qiskit offers visibility into the output of transpilation.

## Put it all together

Now let's take a simple qasm program and run it.

In [ ]:
# importing Qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import BasicAer, IBMQ, execute

backend = BasicAer.get_backend('qasm_simulator') # run on local simulator by default

# Uncomment the following lines to run on a real device
# IBMQ.load_accounts()
# from qiskit.providers.ibmq import least_busy
# backend = least_busy(IBMQ.backends(operational=True, simulator=False))
# print("the best backend is " + backend.name())

In [ ]:
# You might get your qasm by opening a file instead
qasm_source = """// yiqing line (one of many possible)
OPENQASM 2.0;
include "qelib1.inc";
qreg q[3];
creg c[3];
h q[0];
h q[1];
h q[2];
barrier q[0],q[1],q[2];
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
"""

In [ ]:
# Create circuit
circ = QuantumCircuit.from_qasm_str(qasm_source)
from qiskit.compiler import transpile

# View representation of the circuit from a trial transpilation
# When the job executes, transpilation will take place again automatically
print(transpile(circ, backend=backend))

In [ ]:
# Number of shots to run the program (experiment); maximum is 8192 shots.
shots = 256
# Maximum number of credits to spend on executions.
max_credits = 6
job_exp = execute(circ, backend=backend, shots=shots,
                  max_credits=max_credits, memory=True)
from qiskit.tools.monitor import job_monitor

# Monitor the job ... the status show below as output will change as the job progresses
job_monitor(job_exp)

In [ ]:
# See the entire result expressed as a dictionary
result_exp = job_exp.result()
print(result_exp)

In [ ]:
# Let's just look at the result data from the result without the other overhead
print(result_exp.data())

In [ ]:
# A function to generate a little database of the result data
def csv_str(description, sorted_keys, sorted_counts):
    """Generate a cvs as a string from sorted keys and sorted counts"""
    csv = []
    csv.append(description)
    keys = ""
    for key in sorted_keys:
        keys += key + ';'
    csv.append(keys)
    counts = ""
    for count in sorted_counts:
        counts += str(count) + ';'
    csv.append(counts)
    return csv

In [ ]:
# Get the counts and keys from the result and convert to a csv
counts_exp = result_exp.get_counts(circ)
sorted_keys = sorted(counts_exp.keys())
sorted_counts = []
for i in sorted_keys:
    sorted_counts.append(counts_exp.get(i))

import datetime
output = csv_str(str(backend) + ' ' + datetime.datetime.now().isoformat(),
                 sorted_keys, sorted_counts)
for line in output:
    print(line)
